In [1]:
import sys
import os
sys.path.insert(0, os.path.abspath(".."))

from gen.curriculum import get_cbc_grouped_questions
from gen.utils import *

In [2]:
grouped_questions = get_cbc_grouped_questions(
    strand_ids=[2,4,1,6,3,8,9],
    question_count=10,
    bloom_skill_count=2,
    is_debug=True,
)


✅ Question breakdown written to /Users/melaniefayne/Desktop/mtihani/mtihani_api/mtihaniapi/gen/output/question_breakdown.json. Total: 7


In [3]:
OPENAI_LLM_4O = ChatOpenAI(
    model_name="gpt-4o",
    temperature=0.1,
    max_tokens=10240,
    openai_api_key=OPENAI_API_KEY,
)

OPENAI_LLM_4_TURBO = ChatOpenAI(
    model_name="gpt-4-turbo",
    temperature=0.1,
    max_tokens=4096,
    openai_api_key=OPENAI_API_KEY,
)

selected_model = OPENAI_LLM_4_TURBO

all_question_list = generate_llm_question_list(
    grouped_question_data=grouped_questions,
    is_debug=True,
    llm=selected_model,
)

# If there was a generation error
if not isinstance(all_question_list, list):
    print(all_question_list)


Acids, Bases and Indicators =========
📝 Input token count (gpt-4-turbo): 720
📦 Raw LLM output:
 content='[\n    {\n        "question": "In a school science club, Kamau and his friends decide to use beetroot to create a natural pH indicator for their science fair project; they extract the juice and test it with various household substances. Explain how the color change in the beetroot extract can help determine the pH of the substances.",\n        "expected_answer": "The beetroot extract changes color depending on the pH level of the substance it comes into contact with; acidic substances cause it to turn pink or red, while basic substances make it turn yellow or green, thus indicating the substance\'s pH."\n    },\n    {\n        "question": "During a home science experiment, Amina observes that lemon juice turns blue litmus paper red and baking soda solution turns red litmus paper blue. Analyze why these changes in color occur and what they signify about the nature of lemon juice and

In [4]:
exam_questions = get_db_question_objects(
    all_question_list=all_question_list,
    is_debug=True,
)


✅ Question list to /Users/melaniefayne/Desktop/mtihani/mtihani_api/mtihaniapi/gen/output/question_list.json. Total: 10


In [5]:
OUTPUT_FILE = os.path.join(
    BASE_DIR, "output/model_comparisons", f"{selected_model.model_name}_EXAM.txt")

def export_exam_questions_to_txt(exam_questions, output_file=OUTPUT_FILE):
    with open(output_file, "w", encoding="utf-8") as f:
        for idx, q in enumerate(exam_questions, start=1):
            f.write(f"\nQuestion {idx}\n")
            f.write("-" * 20 + "\n")
            f.write(f"Grade: {q.get('grade', '')}\n")
            f.write(f"Strand: {q.get('strand', '')}\n")
            f.write(f"Sub Strand: {q.get('sub_strand', '')}\n\n")
            
            # Handle single or multiple bloom skills per question
            bloom_skills = q.get("bloom_skills", [])
            questions = q.get("questions", [])
            answers = q.get("expected_answers", [])
            
            # If your structure is per-skill, per-QA:
            for b_idx, (bloom, ques, ans) in enumerate(zip(bloom_skills, questions, answers), start=1):
                f.write(f"Bloom Skill {b_idx}: {bloom}\n")
                f.write(f"      Q: {ques}\n")
                f.write(f"      A: {ans}\n\n")
            
            # If your structure is single-skill per question, you can do:
            # f.write(f"Bloom Skill 1: {q.get('bloom_skill', '')}\n")
            # f.write(f"      Q: {q.get('description', '')}\n")
            # f.write(f"      A: {q.get('expected_answer', '')}\n\n")
                
        print(f"✅ Exam questions exported to {output_file}")

# Usage:
# exam_questions = [...]  # your processed list of dicts
export_exam_questions_to_txt(exam_questions)

✅ Exam questions exported to /Users/melaniefayne/Desktop/mtihani/mtihani_api/mtihaniapi/gen/output/model_comparisons/gpt-4-turbo_EXAM.txt
